In [1]:
import torch
import cv2
import time
import pytesseract
import re
import numpy as np

In [ ]:
!pip install pytesseract

In [2]:
def detectx (frame, model):
    frame = [frame]
    print(f"[INFO] Detecting. . . ")
    results = model(frame)
    labels, cordinates = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
    return labels, cordinates

In [3]:
def plot_boxes(results, frame,classes):
    
    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]

    print(f"[INFO] Total {n} detections. . . ")
    print(f"[INFO] Looping through all detections. . . ")


    for i in range(n):
        row = cord[i]
        if row[4] >= 0.30:
            print(f"[INFO] Extracting BBox coordinates. . . ")
            x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape) ## BBOx coordniates
            text_d = classes[int(labels[i])]

            coords = [x1,y1,x2,y2]

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(frame, (x1, y1-20), (x2, y1), (0, 255,0), -1) 

    return frame

In [4]:
def main(img_path=None, vid_path=None,vid_out = None):

    print(f"[INFO] Loading model... ")
    model =  torch.hub.load("./yolov5", 'custom', source ='local', path='best.pt',force_reload=True) 

    classes = model.names

    if img_path != None:
        print(f"[INFO] Working with image: {img_path}")
        img_out_name = f"./output/result_{img_path.split('/')[-1]}"

        frame = cv2.imread(img_path)
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        
        results = detectx(frame, model = model)   

        frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)

        frame = plot_boxes(results, frame,classes = classes)
        
        cv2.namedWindow("img_only", cv2.WINDOW_NORMAL)

        while True:
            cv2.imshow("img_only", frame)
            if cv2.waitKey(5) & 0xFF == ord('q'):
                print(f"[INFO] Exiting. . . ")
                cv2.imwrite(f"{img_out_name}",frame)
                break
    cv2.destroyAllWindows()
    

    if vid_path != None:
        print(f"[INFO] Working with video: {vid_path}")

        cap = cv2.VideoCapture(vid_path)

        if vid_out:

            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            codec=cv2.VideoWriter_fourcc(*'MP4V')
            out = cv2.VideoWriter(vid_out, codec, fps, (width, height))

        assert cap.isOpened()
        frame_no = 1

        cv2.namedWindow("vid_out", cv2.WINDOW_NORMAL)
        while True:
            ret, frame = cap.read()
            if ret  and frame_no %1 == 0:
                print(f"[INFO] Working with frame {frame_no} ")

                frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                results = detectx(frame, model = model)
                
                frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)


                frame = plot_boxes(results, frame,classes = classes)
                
                cv2.imshow("vid_out", frame)
                if vid_out:
                    print(f"[INFO] Saving output video. . . ")
                    out.write(frame)

                if cv2.waitKey(5) & 0xFF == ord('q'):
                    break
                frame_no += 1
        
        print(f"[INFO] Clening up. . . ")
        out.release()
        
    cv2.destroyAllWindows()
    

In [6]:
#main(vid_path=r"C:\Users\Jayasuriya\Downloads\WhatsApp Video 2023-05-02 at 4.09.47 PM.mp4",vid_out="PXL_20220913_080309491.mov") ### for custom video
#main(vid_path=0,vid_out="webcam_facemask_result.mp4") #### for webcam
main(img_path=r"C:\Users\Jayasuriya\Downloads\img-39_jpg.rf.a4ba4e9ad51da5bf7e864ca580d09931.jpg") ## for image

[INFO] Loading model... 


YOLOv5  v7.0-72-g064365d Python-3.9.16 torch-1.8.2+cu111 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


[INFO] Working with image: C:\Users\Jayasuriya\Downloads\img-39_jpg.rf.a4ba4e9ad51da5bf7e864ca580d09931.jpg
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[INFO] Exiting. . . 
